In [ ]:
import pandas as pd
import numpy as np
df = pd.read_excel("resources/SENTIMENT DATA.xlsx")

In [ ]:
df.head(2)

In [ ]:
df_c = df['COMMENTS']

In [ ]:
# 모든 문자를 소문자로 바꿔주는 방법
# 방법 1 : pandas 의 str.casefold 메소드를 쓰는 방법
df_c = df_c.str.casefold()
# 방법 2 : 데이터프레임의 row 마다의 문자를 분해해서 lowercase 로 바꿔주고 다시 합치는 방법
# df = df.apply (lambda x : " ".join(x.lower for x in str(x).split()))


In [ ]:
df_c[78]

In [ ]:
# Punctuation 제거 및 숫자제거
# Punctuation 제거 (정규식 사용)
df_c = df_c.str.replace("[^\w\s]",'')
df_c.tail()
# 숫자 제거 (정규식 사용)
df_c = df_c.str.replace("\d+",'')
df_c[78]

In [ ]:
# nltk 라이브러리 호출
# nltk stopwords corpus 호출
from nltk.corpus import stopwords
stop = stopwords.words('english')
len(stop)

In [ ]:
# stop words 제거
df_c = df_c.apply (lambda x : " ".join(x for x in x.split() if x not in stop))

In [ ]:
df_c.head()

In [ ]:
df_c[78]

In [ ]:
# stemming process
from nltk.stem import PorterStemmer
st = PorterStemmer()
df_c = df_c.apply (lambda x : " ".join([st.stem(word) for word in x.split()]))

In [ ]:
df_c.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [ ]:
x = cv.fit_transform(df_c).toarray()
print(len(x))
x_col = cv.get_feature_names()
print(len(x_col))
df_x = pd.DataFrame(x, columns=x_col)
df_x.head()

In [ ]:
y = df["POLARITY"]
y

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split (x,y, test_size =0.2, random_state = 0)

In [ ]:
len(x_train)

In [ ]:
len(x_test)

In [ ]:
x_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf  = LogisticRegression()
clf.fit(x_train, y_train)

In [ ]:
def show_most_informative_features(vectorizer, clf, n=25):
    d = {}
    a=0
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    print("\t\t\tNegative\t\t\t\t\t\tPositive")
    print("________________________________________________________________________________________________")
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print("\t%.4f\t%-15s\t\t\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))
        d[a]=fn_1
        a+=1
    
show_most_informative_features(cv,clf)

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import classification_report as cr
print (cr(y_test,y_pred))

## Model Deployment 

In [ ]:
df_copy = pd.read_excel('C:\\predict_comment.xlsx')

In [ ]:
df_copy['COMMENT'] = df_copy['COMMENT'].str.casefold()
# TOLOWER

df_copy['COMMENT'] = df_copy['COMMENT'].str.replace('[^\w\s]','')
# Remove punctuations

df_copy["COMMENT"]=df_copy["COMMENT"].str.replace('\d+','')
# Remove digits

from nltk.corpus import stopwords
stop = stopwords.words('english')
df_copy['COMMENT'] = df_copy['COMMENT'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df_copy['COMMENT'].head()
# Remove Stop Words

from nltk.stem import PorterStemmer
st = PorterStemmer()
df_copy['COMMENT'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
# Stemming

In [ ]:
feature_list = cv.get_feature_names()

In [ ]:
vect= CountVectorizer(vocabulary = feature_list)
X = vect.transform(df_copy['COMMENT']).toarray() 

In [ ]:
y_pred = clf.predict(X)

In [ ]:
y_pred